In [56]:
%load_ext autoreload
%autoreload 2
from __future__ import print_function
import os,sys
sys.path.append('../')

## Math and dataFrame
import numpy as np
import pandas as pd
import scipy
import seaborn as sns

import matplotlib.pyplot as plt

#ML
from sklearn.model_selection import cross_val_predict, StratifiedKFold, KFold, train_test_split
from sklearn.metrics import log_loss, accuracy_score, f1_score, confusion_matrix
from keras.callbacks import ModelCheckpoint

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
label_cols = ['toxic', 'severe_toxic', 'obscene',  'threat', 'insult', 'identity_hate']
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

train['dirtyness'] = train.apply(lambda x: x.iloc[2::].sum(), axis = 1)
test['dirtyness'] = test.apply(lambda x: x.iloc[2::].sum(), axis = 1)

COMMENT = 'comment_text'
train[COMMENT].fillna("unknown", inplace=True)
test[COMMENT].fillna("unknown", inplace=True)

print("train set len ", len(train) )
print("test set len ", len(test) )
print("clean samples", len(train[train['dirtyness'] == 0]))
print("toxic samples", len(train[train['dirtyness'] != 0]))

train set len  159571
test set len  153164
clean samples 143346
toxic samples 16225


In [5]:
# tokenize
max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2

#preprocessing
raw_text = np.hstack([train.comment_text.str.lower(), test.comment_text.str.lower()])
tok_raw = Tokenizer(num_words=max_features)
tok_raw.fit_on_texts(raw_text)

train["seq"] = tok_raw.texts_to_sequences(train.comment_text.str.lower())
test["seq"] = tok_raw.texts_to_sequences(test.comment_text.str.lower())

#sequence.pad_sequences(train['comment_text'].values, maxlen=maxlen)
#test['comment_text'] = sequence.pad_sequences(test['comment_text'], maxlen=maxlen)
#print('x_train shape:', train['comment_text'].values.shape)
#print('x_test shape:', test['comment_text'].values.shape)

In [7]:
# pad
#train["seq"].apply(lambda x: len(x)).describe()
train["seq_pad"] = train["seq"].apply(lambda x, maxlen: sequence.pad_sequences([x], maxlen=maxlen)[0], args = [maxlen])
test["seq_pad"] = test["seq"].apply(lambda x, maxlen: sequence.pad_sequences([x], maxlen=maxlen)[0], args = [maxlen])


In [8]:
#build model
def buildModel():
    model = Sequential()

    # we start off with an efficient embedding layer which maps
    # our vocab indices into embedding_dims dimensions
    model.add(Embedding(max_features,
                        embedding_dims,
                        input_length=maxlen))
    model.add(Dropout(0.2))

    # we add a Convolution1D, which will learn filters
    # word group filters of size filter_length:
    model.add(Conv1D(filters,
                     kernel_size,
                     padding='valid',
                     activation='relu',
                     strides=1))
    # we use max pooling:
    model.add(GlobalMaxPooling1D())

    # We add a vanilla hidden layer:
    model.add(Dense(hidden_dims))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))

    # We project onto a single unit output layer, and squash it with a sigmoid:
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    # define metrics and compile model
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

modeldict = {}
for lc in label_cols:
    modeldict[lc] = buildModel()
    modeldict[lc].summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 400, 50)           250000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 400, 50)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 398, 250)          37750     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 250)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 250)               62750     
_________________________________________________________________
dropout_2 (Dropout)          (None, 250)               0         
_________________________________________________________________
activation_1 (Activation)    (None, 250)               0         
__________

In [9]:
# load fe , we will use resample to balance the labels
from models.FeatureExtraction import FeatureExtraction

fe = FeatureExtraction()

trainOrig = np.array(train['seq_pad'].tolist())
assert(trainOrig.shape == (len(train), maxlen))

label0 - 144277
label1 - 15294
Resample label 1


In [26]:
#define F1 metric
import keras
#fit
for lc in label_cols:
    # train test split
    trn_re, label_re = fe.reSample( scipy.sparse.csr_matrix(trainOrig) , y = train[lc])
    x_train, x_val, y_train, y_val = train_test_split(trn_re, label_re, test_size=0.33, random_state=42)
    
    # add check point
    filepath= lc + "_weights_best.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]

    modeldict[lc].fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_val, y_val), 
          callbacks=callbacks_list
             )

label0 - 144277
label1 - 15294
Resample label 1
Train on 193331 samples, validate on 95223 samples
Epoch 1/2
193312/193331 [============================>.] - ETA: 0s - loss: 0.0686 - acc: 0.9763
Epoch 00001: val_acc improved from -inf to 0.97381, saving model to toxic_weights_best.hdf5
193331/193331 [==============================] - 719s 4ms/step - loss: 0.0686 - acc: 0.9763 - val_loss: 0.0775 - val_acc: 0.9738
Epoch 2/2
193312/193331 [============================>.] - ETA: 0s - loss: 0.0526 - acc: 0.9822
Epoch 00002: val_acc improved from 0.97381 to 0.97494, saving model to toxic_weights_best.hdf5
193331/193331 [==============================] - 714s 4ms/step - loss: 0.0526 - acc: 0.9822 - val_loss: 0.0768 - val_acc: 0.9749
label0 - 157976
label1 - 1595
Resample label 1
Train on 211687 samples, validate on 104265 samples
Epoch 1/2
211680/211687 [============================>.] - ETA: 0s - loss: 0.0550 - acc: 0.9852
Epoch 00001: val_acc improved from -inf to 0.99214, saving model to s

In [58]:
# implement Trained Model
from models.TrainedModel import TrainedModel
import modelDB

class TrainedModelCNNEmbedding(TrainedModel):
    def __init__(self, md = None):
        super(TrainedModelCNNEmbedding, self).__init__(md)
    
    def predict(self, test, **kwargs):
        print ("Predict using Model: ")
        for i in self.md.keys():  
            print( "{} - {}".format(i, type(self.md[i]).__name__ ) )
            
        res = {}
        for topic, submd in zip( self.md.keys(), self.md.values() ):
            print( " predicitng topic {}".format(topic))
            res[topic] = zip(*submd.predict(test, batch_size = 1024))[0] #obtain proba
        
        #important to keep the order as required by the submission file
        testid = kwargs['testid'] 
        #print (testid)
        dfres = pd.DataFrame(res)
        dfres['id'] = testid
        
        #print(dfres.shape)
        assert(dfres.shape[0] == test.shape[0])
        
        #reshape to submission file format
        dfres = dfres[['id', 'toxic','severe_toxic','obscene','threat','insult','identity_hate']]
        
        return dfres
    
    def _save(self, mDB, nameKey, modelpath, **kwargs):
        '''
        save model into model data base
        :mDB: meta data frame storing all models info
        :nameKey: unique identifier for each saved model
        :modelpath: subdir inside modelDB dir, e.g. if modelDB is /root/modelDB, then modelpath is /cnn
        '''
        print("Saving model")
        import datetime
        
        #save a dict (topic => ('modelname', 'weightname')) into db
        model_saved_toDB = {}
        for topic, submd in zip( self.md.keys(), self.md.values() ):
            #this is the file we will save model to
            mdname = os.path.join(modelpath, nameKey + '_'+ topic +'_' + '.sav' )
            mdnameAbs = os.path.join(modelDB.MODEL_DB_ROOT, mdname )

            #this is the file we will save weights to
            weightName = os.path.join(modelpath, nameKey +  '_'+ topic + '_' + '_weights.h5' )
            weightNameAbs = os.path.join( modelDB.MODEL_DB_ROOT, weightName )

            #convert md to json and save to file
            print(" Saving model {}".format(mdname) )
            model_json = submd.to_json()
            with open(mdnameAbs, "w") as json_file:
                json_file.write(model_json)

            # serialize weights to HDF5
            print(" Saving weights {}".format(weightName) )
            submd.save_weights(weightNameAbs)
    
            model_saved_toDB[topic] = (mdname, weightName)
            
        
        #db schema
        # 'modelName', type {rnn, cnn, rf}, date, model
        print( "Info: custom saving options" )
        for i in kwargs:
            print("{} - {}".format(i , kwargs[i]))
            
        import json
        newRow = pd.DataFrame({
            'modelName': [nameKey],
            'type': kwargs['modelType'],
            'subType': kwargs['modelSubType'],
            'date': datetime.datetime.now().strftime("%Y-%m-%d"),
            'model' : json.dumps(model_saved_toDB)
        }
        )
        
        #add a new row 
        #mDB = pd.concat([mDB, newRow])
        mDB = mDB.append(newRow, ignore_index = True)
        display(mDB)

        return mDB
    
    def load(self, mDB, nameKey, modelpath):
        '''
        :mDB: meta data frame storing all models info
        :nameKey: unique identifier for each saved model
        :modelpath: subdir inside modelDB dir, e.g. if modelDB is /root/modelDB, then modelpath is /cnn
        '''
        from keras.models import model_from_json
        import json
        
        if (mDB.empty) or mDB[ mDB['modelName'] == nameKey ].empty:
            raise VaueError("Model name does not exist")
        print("loadModel")
        #the saved model is of format: dict (topic => ('modelname', 'weightname')) into db
        assert(len(mDB[mDB['modelName'] == nameKey]) == 1)
        saved_model_inDB = json.loads(mDB[mDB['modelName'] == nameKey].iloc[0]['model'])
        
        print("saved json string representing the model is {}".format(saved_model_inDB))
        loaded_model = {}
        for topic, mdPointer in saved_model_inDB.iteritems():
            print(" load model for topic {}".format(topic))
            mdname = mdPointer[0]
            mdnameAbs = os.path.join(modelDB.MODEL_DB_ROOT, mdname )
            print("  model file in {}".format(mdnameAbs))

            #this is the file we will save weights to
            weightName = mdPointer[1]
            weightNameAbs = os.path.join( modelDB.MODEL_DB_ROOT, weightName )
            print("  weight file in {}".format(weightNameAbs))
            
            with open(mdnameAbs, 'r') as json_file:
                tmpModel_json = json_file.read()
            tmpModel = model_from_json(tmpModel_json)
            #load weights
            tmpModel.load_weights(weightNameAbs)
            
            #assign the model into loaded model dict
            loaded_model[topic] = tmpModel
        
        assert(self.md is None)
        self.setModel(loaded_model)
        return self



In [ ]:
# #Utest test predict
# myCNN1d = TrainedModelCNNEmbedding(modeldict)
# dfres = myCNN1d.predict(trainOrig[0:1000, :], testid = train['id'])
# display(dfres.head(20))
# display(train.head(20)[['id', 'toxic','severe_toxic','obscene','threat','insult','identity_hate']])

# #Utest test save
# mdDB = pd.read_json("../modelDB/modelMetaDB.json")
# modelpath = 'cnn/' # NOTE: this is relative to the modelDB path
# print("current modelDB")
# display(mdDB)

# mdDB = myCNN1d.save(mdDB, 'CNN1d-Embedding_Random-OneperClass', modelpath, modelType = 'CNN1d', modelSubType = 'Embedding_Random')
# #mdDB.to_json("../modelDB/modelMetaDB.json")

# #Utest test load
# loadedCNN = TrainedModelCNNEmbedding().load( mdDB, 'CNN1d-Embedding_Random-OneperClass', modelpath)
# for i in loadedCNN.md.keys():  
#     print( "{} - {}".format(i, loadedCNN.md[i] ) )
# dfres_loaded = loadedCNN.predict(trainOrig[0:1000, :], testid = train['id'])
# assert(dfres_loaded.equals(dfres))

In [ ]:
#forward pass to inference
testOrig = np.array(test['seq_pad'].tolist())

myCNN1d = TrainedModelCNNEmbedding(modeldict)

dfres = myCNN1d.predict(testOrig, testid = test['id'])

In [46]:
#save
myCNN1d = TrainedModelCNNEmbedding(modeldict)

mdDB = pd.read_json("../modelDB/modelMetaDB.json")
modelpath = 'cnn/' # NOTE: this is relative to the modelDB path
print("current modelDB")
display(mdDB)

mdDB = myCNN1d.save(mdDB, 'CNN1d-Embedding_Random-OneperClass', modelpath, modelType = 'CNN1d', modelSubType = 'Embedding_Random')
mdDB.to_json("../modelDB/modelMetaDB.json")


current modelDB


,date,model,modelName,subType,type,weights
0,1970-01-01 00:25:18.220800,cnn/utestModel.sav,utestModel,TfIdf,testType,NaN
1,2018-02-12 00:00:00.000000,"{""severe_toxic"": [""cnn/CNN1d-Embedding_Random-...",CNN1d-Embedding_Random-OneperClass,Embedding_Random,CNN1d,NaN


Saving model
 Saving model cnn/CNN1d-Embedding_Random-OneperClass_severe_toxic_.sav
 Saving weights cnn/CNN1d-Embedding_Random-OneperClass_severe_toxic__weights.h5
 Saving model cnn/CNN1d-Embedding_Random-OneperClass_identity_hate_.sav
 Saving weights cnn/CNN1d-Embedding_Random-OneperClass_identity_hate__weights.h5
 Saving model cnn/CNN1d-Embedding_Random-OneperClass_obscene_.sav
 Saving weights cnn/CNN1d-Embedding_Random-OneperClass_obscene__weights.h5
 Saving model cnn/CNN1d-Embedding_Random-OneperClass_insult_.sav
 Saving weights cnn/CNN1d-Embedding_Random-OneperClass_insult__weights.h5
 Saving model cnn/CNN1d-Embedding_Random-OneperClass_threat_.sav
 Saving weights cnn/CNN1d-Embedding_Random-OneperClass_threat__weights.h5
 Saving model cnn/CNN1d-Embedding_Random-OneperClass_toxic_.sav
 Saving weights cnn/CNN1d-Embedding_Random-OneperClass_toxic__weights.h5
Info: custom saving options
modelSubType - Embedding_Random
modelType - CNN1d


,date,model,modelName,subType,type,weights
0,1970-01-01 00:25:18.220800,cnn/utestModel.sav,utestModel,TfIdf,testType,NaN
1,2018-02-12,"{""severe_toxic"": [""cnn/CNN1d-Embedding_Random-...",CNN1d-Embedding_Random-OneperClass,Embedding_Random,CNN1d,NaN
